In [56]:
import pandas as pd
from kafka import KafkaProducer
from datetime import datetime
import time
import random
import numpy as np

In [57]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import base64, expr

spark = SparkSession.builder.getOrCreate()

In [58]:
filepath = "IRIS.csv"
# flower_df = pd.read_csv(filepath)
df = spark.read.option('header','true').csv(filepath)
df.show()

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    species|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|          3|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|           5|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|           5|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9|        3.1|         1.5|        0.1|Iris-setosa|
|         5.4|        3.7|         1.5|        0.2|Iris-setosa|
|         4.8|        3.4|         1.6|        0.2|Iris-setosa|
|         4.8|          3|         1.4| 

In [59]:
secret_key = 'Thisguyisgonnagetsomeact'
df = df.withColumn('SSN_encrypted', base64(expr(f"aes_encrypt(species, '{secret_key}', 'ECB')")))
# df_encrypted.printSchema()
# df_encrypted.show()

In [60]:
df_encrypted = df_encrypted.withColumn('SSN_decrypted', expr(f"aes_decrypt(unbase64(SSN_encrypted), '{secret_key}', 'ECB')").cast('string'))
df_encrypted.printSchema()
df_encrypted.select(['SSN_decrypted', 'species']).show()

root
 |-- sepal_length: string (nullable = true)
 |-- sepal_width: string (nullable = true)
 |-- petal_length: string (nullable = true)
 |-- petal_width: string (nullable = true)
 |-- species: string (nullable = true)
 |-- SSN_encrypted: string (nullable = true)
 |-- SSN_decrypted: string (nullable = true)

+-------------+-----------+
|SSN_decrypted|    species|
+-------------+-----------+
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
|  Iris-setosa|Iris-setosa|
+-------------+-----------+
only showing top 20

In [63]:


# pip install kafka-python

KAFKA_TOPIC_NAME_CONS = "Topic"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'

secret_key = 'Thisguyisgonnagetsomeact'


if __name__ == "__main__":
    print("Kafka Producer Application Started ... ")

    kafka_producer_obj = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS_CONS,
                                       value_serializer=lambda x: x.encode('utf-8'))
    
    filepath = "IRIS.csv"
    # flower_df = pd.read_csv(filepath)
    flower_df = spark.read.option('header','true').csv(filepath)
    
    df_encrypted = flower_df.withColumn('species', base64(expr(f"aes_encrypt(species, '{secret_key}', 'ECB')")))
    
    # df_encrypted['order_id'] = np.arange(len(flower_df))

    
    # flower_list = df_encrypted.to_dict(orient="records")
    rows = df_encrypted.collect()
    flower_list = [row.asDict() for row in rows]

    message_list = []
    message = None
    for message in flower_list:
        
        message_fields_value_list = []
               
        # message_fields_value_list.append(message["order_id"])
        message_fields_value_list.append(message["sepal_length"])
        message_fields_value_list.append(message["sepal_width"])
        message_fields_value_list.append(message["petal_length"])
        message_fields_value_list.append(message["petal_width"])
        message_fields_value_list.append(message["species"])
        # message_fields_value_list.append(message[""])

        message = ','.join(str(v) for v in message_fields_value_list)
        print("Message Type: ", type(message))
        print("Message: ", message)
        kafka_producer_obj.send(KAFKA_TOPIC_NAME_CONS, message)
        time.sleep(1)


    print("Kafka Producer Application Completed. ")

Kafka Producer Application Started ... 
Message Type:  <class 'str'>
Message:  5.1,3.5,1.4,0.2,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  4.9,3,1.4,0.2,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  4.7,3.2,1.3,0.2,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  4.6,3.1,1.5,0.2,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  5,3.6,1.4,0.2,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  5.4,3.9,1.7,0.4,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  4.6,3.4,1.4,0.3,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  5,3.4,1.5,0.2,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  4.4,2.9,1.4,0.2,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  4.9,3.1,1.5,0.1,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  5.4,3.7,1.5,0.2,MiKAg50XuuY+/V0O04LOvQ==
Message Type:  <class 'str'>
Message:  4.8,3.4,1.6,0.2,MiKAg50XuuY+/V0O04LOvQ==
Messag